In [19]:
import pandas as pd
import numpy as np
import send_to_racing
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
import time
from keras import regularizers
from keras_tuner import HyperParameters, GridSearch, RandomSearch
from keras.layers import Dropout
from keras.callbacks import EarlyStopping

In [20]:
# Prepare your data
bbox_data = np.array(send_to_racing.bbox)
bbox_data = bbox_data[:432, 3:5]
depth = np.array(send_to_racing.depth_values).reshape(441, 1)
test_data = np.array(send_to_racing.bbox)[432:, 3:5]

In [21]:
# Define the model building function
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units_0', min_value=16, max_value=128, step=16),
                    input_shape=(2,),
                    activation='relu',
                    kernel_regularizer=regularizers.l2(hp.Choice('l2_0', values=[0.0, 0.01, 0.001]))))

    for i in range(hp.Int('num_layers', 0, 2)):
        model.add(Dense(units=hp.Int(f'units_{i+1}', min_value=16, max_value=128, step=16),
                        activation='relu',
                        kernel_regularizer=regularizers.l2(hp.Choice(f'l2_{i+1}', values=[0.0, 0.01, 0.001]))))
        
        model.add(Dropout(rate=hp.Float(f'dropout_rate_{i}', min_value=0.0, max_value=0.5, step=0.1)))

    model.add(Dense(1, activation='relu'))

    model.compile(optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                  loss='mse',
                  metrics=['mse'])
    return model

In [22]:
# Set up Keras Tuner with Random Search
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=50,
    executions_per_trial=3,
    directory='my_dir',
    project_name='hyperparameter_tuning'
)

In [23]:
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
# Search for the best hyperparameters
tuner.search(
    bbox_data, depth,
    validation_split=0.40,
    batch_size=10,
    epochs=700,
    shuffle=True,
    verbose=1
)

Trial 50 Complete [00h 01m 22s]
val_loss: 1.9455689589182537

Best val_loss So Far: 0.12005000064770381
Total elapsed time: 01h 17m 46s


In [25]:
# Retrieve the best model and hyperparameters
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters()[0]

print("Best hyperparameters: ", best_hyperparameters.values)

Best hyperparameters:  {'units_0': 128, 'l2_0': 0.0, 'num_layers': 1, 'learning_rate': 0.0011171470937487741, 'units_1': 64, 'l2_1': 0.0, 'dropout_rate_0': 0.2, 'units_2': 112, 'l2_2': 0.001, 'dropout_rate_1': 0.1}
